# Zero Field Splitting Calculation using WAVECAR

I think PyZFS has all the functionality to calculate ZFS. All I have to do is to create a proper wavefunction loader class that can extract required information from WAVECAR (from gamma point only calculations) and give me the ZFS value.

In [ ]:
pwd

## ZFS for O2 molecule

Let's calculate the ZFS for O2 molecule from given data.

In [ ]:
cd /home/thsim21/Shibu/Project/code_development/ext_lib/pyzfs/examples/o2_qe_hdf5

In [ ]:
from pyzfs.common.wfc.qeh5loader import QEHDF5WavefunctionLoader
from pyzfs.zfs.main import ZFSCalculation

In [ ]:
# Construct wavefunction loader
wfcloader = QEHDF5WavefunctionLoader(fftgrid='wave')

In [ ]:
# Construct ZFSCalculation
zfscalc = ZFSCalculation(wfcloader=wfcloader)

In [ ]:
# Perform ZFS calculation
zfscalc.solve()

## ZFS Calculation from WAVECAR

In [ ]:
cd /home/thsim21/Shibu/Project/code_development/ext_lib/pyzfs/examples/test

In [ ]:
import numpy as np

from pyzfs.common.wfc.baseloader import WavefunctionLoader
from pyzfs.common.cell import Cell
from pyzfs.common.ft import FourierTransform
from pyzfs.common.wfc.wavefunction import Wavefunction
from pyzfs.common.parallel import mpiroot

In [ ]:
class VaspWavefunctionLoader(WavefunctionLoader):

    def scan(self):
        super(VaspWavefunctionLoader, self).scan()

        from ase.io import read
        # from ...common.misc import vaspwfc

        # Read cell from POSCAR file
        ase_cell = read("POSCAR")
        cell = Cell(ase_cell)

        # Read wfc from WAVECAR file
        self.wavecar = vaspwfc()
        ft = FourierTransform(*self.wavecar._ngrid)
        nspin, nkpts, nbands,_ = self.wavecar._occs.shape
        assert nspin == 2 and nkpts == 1

        # Get band indices (starting from 1) witt significant occupations
        iuorbs = np.where(self.wavecar._occs[0, 0] > 0.8)[0] + 1
        idorbs = np.where(self.wavecar._occs[1, 0] > 0.8)[0] + 1

        nuorbs = len(iuorbs)
        ndorbs = len(idorbs)
        norbs = nuorbs + ndorbs

        iorb_sb_map = list(
            ("up", iuorbs[iorb]) if iorb < nuorbs
            else ("down", idorbs[iorb - nuorbs])
            for iorb in range(norbs)
        )

        iorb_fname_map = ["WAVECAR"] * norbs
        self.wfc = Wavefunction(cell=cell, ft=ft, nuorbs=nuorbs, ndorbs=ndorbs,
                                iorb_sb_map=iorb_sb_map, iorb_fname_map=iorb_fname_map)

    def load(self, iorbs, sdm=None):
        super(VaspWavefunctionLoader, self).load(iorbs, sdm, sdm)

        counter = 0
        for iorb in iorbs:
            spin, band = self.wfc.iorb_sb_map[iorb]
            psir = self.wavecar.wfc_r(
                ispin=1 if spin == "up" else 2, iband=band, gamma=True
            )
            self.wfc.set_psir(iorb, psir)

            counter += 1
            if counter >= len(iorbs) // 10:
                if mpiroot:
                    print("........")
                counter = 0

In [ ]:
from pymatgen.io.vasp.outputs import Wavecar

In [ ]:
wavecar1 = Wavecar('WAVECAR')

In [ ]:
from ase.io import read
ase_cell = read("POSCAR")
cell = Cell(ase_cell)

In [1]:
from pyzfs.common.wfc.vasploader import VaspWavefunctionLoader
from pyzfs.zfs.main import ZFSCalculation
wfcloader = VaspWavefunctionLoader()
#zfscalc = ZFSCalculation(wfcloader=wfcloader)
# Perform ZFS calculation
#zfscalc.solve()


VaspWavefunctionLoader: scanning current working directory "/home/thsim21/Shibu/Project/code_development/ext_lib/pyzfs/examples/test"...

   nuwfcs = 191, ndwfcs = 189, nwfcs = 380
     spin = up     band = 1     file = WAVECAR
     spin = up     band = 2     file = WAVECAR
     spin = up     band = 3     file = WAVECAR
     spin = up     band = 4     file = WAVECAR
     spin = up     band = 5     file = WAVECAR
     spin = up     band = 6     file = WAVECAR
     spin = up     band = 7     file = WAVECAR
     spin = up     band = 8     file = WAVECAR
     spin = up     band = 9     file = WAVECAR
     spin = up     band = 10     file = WAVECAR
     spin = up     band = 11     file = WAVECAR
     spin = up     band = 12     file = WAVECAR
     spin = up     band = 13     file = WAVECAR
     spin = up     band = 14     file = WAVECAR
     spin = up     band = 15     file = WAVECAR
     spin = up     band = 16     file = WAVECAR
     spin = up     band = 17     file = WAVECAR
     spin =

In [2]:
zfscalc = ZFSCalculation(wfcloader=wfcloader)
# Perform ZFS calculation
#zfscalc.solve()

  
  
  Zero Field Splitting Calculation Created...
  
  
    ProcessGrid (square) info:
    rank -> (irow, icol) mapping:
    [[0 0]]
    (irow, icol) -> rank mapping:
    [[0]]
    rank  onroot  nrow  ncol  size  irow  icol  is_active  rowcomm_rank  colcomm_rank
      0    True      1    1    1    0    0    True    0    0
  
  Creating I array...
  
    DistributedMatrix info: I
    irow  icol  nrow  ncol  m  mloc  mstart  mend  n  nloc  nstart  nend  val.shape
      0    0    1    1    380    380    0    380    380    380    0    380     (380, 380, 6)
    Index map:
    [[[  0 380 380   0 380 380]]]
  
  Memory usage (on process 0):
    iorb_psir_map 0.00 MB
    iorb_rhog_map 0.00 MB
    I          6.61 MB
  Total memory usage (on process 0): 134.77 MB
  


In [3]:
zfscalc.solve()

  
  VaspWavefunctionLoader: loading orbitals into memory... (memory mode: "critical")
  
  ........
  ........
  ........
  ........
  ........
  ........
  ........
  ........
  ........
  ........
  
  Memory usage (on process 0):
    iorb_psir_map 247.79 MB
    iorb_rhog_map 0.00 MB
    I          6.61 MB
  Total memory usage (on process 0): 359.69 MB
  
  
  Computing dipole-dipole interaction tensor in G space...
  
  
  Memory usage (on process 0):
    iorb_psir_map 247.79 MB
    iorb_rhog_map 0.00 MB
    ddig       1.96 MB
    I          6.61 MB
  Total memory usage (on process 0): 361.64 MB
  
  
  Iterating over pairs...
  
  (process 0) 724 pairs (1%) computed in 4.329954s...
  (process 0) 1448 pairs (2%) computed in 3.871650s...
  (process 0) 2172 pairs (3%) computed in 3.765174s...
  (process 0) 2896 pairs (4%) computed in 3.754349s...
  (process 0) 3620 pairs (5%) computed in 3.775186s...
  (process 0) 4344 pairs (6%) computed in 3.900628s...
  (process 0) 5068 pairs (7%)

In [ ]:
from pyzfs.VaspBandUnfolding.vaspwfc import vaspwfc
vaspwfc??

In [ ]:
from pymatgen.io.vasp.outputs import Wavecar
class vaspwfc:
    
    def __init__(self, filename="WAVECAR"):
        self.all_info = Wavecar(filename)
        self._ngrid = self.all_info.ng
        #self.band_energy = np.array(self.all_info.band_energy)
        self._occs = self.band_energy_to_occs(self.all_info.band_energy)
        
    
    def band_energy_to_occs(self, band_energy):
        """
        Convert the band energy information extracted from WAVECAR
        using pymatgen.io.vasp.outputs to occs to be used for ZFS calculations.

        Parameters : 
        ============
        band_energy : np.ndarray (nspin x nkpts x nbands x 3)

        Output :
        ========
        occs : np.ndarray (nspin x nkpts x nbands)
        """
        occs = []
        band_energy = np.array(band_energy)
        nspin, nkpts, nbands, dim = band_energy.shape
        for i in range(int(nspin)):
            k_ops = []
            for j in range(int(nkpts)):
                k_ops.append(band_energy[i][j][:, dim-1])
            occs.append(np.array(k_ops))
        return np.array(occs)
    

In [ ]:
from ase.io import read
ase_cell = read("POSCAR")
cell = Cell(ase_cell)

wavecar = vaspwfc()
ft = FourierTransform(*wavecar._ngrid)
nspin, nkpts, nbands= wavecar._occs.shape
assert nspin == 2 and nkpts == 1

iuorbs = np.where(wavecar._occs[0, 0] > 0.8)[0] + 1
idorbs = np.where(wavecar._occs[1, 0] > 0.8)[0] + 1

nuorbs = len(iuorbs)
ndorbs = len(idorbs)
norbs = nuorbs + ndorbs

iorb_sb_map = list(
    ("up", iuorbs[iorb]) if iorb < nuorbs
    else ("down", idorbs[iorb - nuorbs])
    for iorb in range(norbs)
)

iorb_fname_map = ["WAVECAR"] * norbs
wfc = Wavefunction(cell=cell, ft=ft, nuorbs=nuorbs, ndorbs=ndorbs,iorb_sb_map=iorb_sb_map, iorb_fname_map=iorb_fname_map)

In [ ]:
mesh = Wavecar('WAVECAR').fft_mesh(0,100)#(kpoint, band)
evals = np.fft.ifftn(mesh)

In [5]:
from pyzfs.VaspBandUnfolding.vaspwfc import vaspwfc
pswfc = vaspwfc('WAVECAR')
# KS orbital in real space, double the size of the FT grid
phi = pswfc.get_ps_wfc(ikpt=1, iband=27, ngrid=pswfc._ngrid)

In [ ]:
evals[0][0][0], mesh[0][0][0]

In [9]:
pswfc._occs.shape

(2, 1, 320)

In [ ]:
import mycode

In [ ]:
np.where(wavecar._occs[0, 0] > 0.8)[0] + 1

In [ ]:
wavecar1.__dict__.keys()

In [ ]:
np.array(wavecar1.__dict__['band_energy']).shape

In [ ]:
np.array(wavecar1.band_energy).shape

In [ ]:
np.array([np.array(wavecar1.band_energy)[0][0][:, 2]]).shape, np.array([np.array(wavecar1.band_energy)[1][0][:, 2]]).shape

In [ ]:
np.array([np.array([np.array(wavecar1.band_energy)[0][0][:, 2]]), np.array([np.array(wavecar1.band_energy)[1][0][:, 2]])]).shape

In [ ]:
def band_energy_to_occs(band_energy):
    """
    Convert the band energy information extracted from WAVECAR
    using pymatgen.io.vasp.outputs to occs to be used for ZFS calculations.
    
    Parameters : 
    ============
    band_energy : np.ndarray (nspin x nkpts x nbands x 3)
    
    Output :
    ========
    occs : np.ndarray (nspin x nkpts x nbands)
    """
    occs = []
    band_energy = np.array(band_energy)
    nspin, nkpts, nbands, dim = band_energy.shape
    for i in range(int(nspin)):
        k_ops = []
        for j in range(int(nkpts)):
            k_ops.append(band_energy[i][j][:, dim-1])
        occs.append(np.array(k_ops))
    return np.array(occs)

In [ ]:
for i in range(1):
    print(i)

In [ ]:
wavecar2 = Wavecar("/home/thsim21/Shibu/Project/code_development/ext_lib/pyzfs/examples/divac_sic_vasp/WAVECAR")
band_energy_to_occs(wavecar2.band_energy).shape

In [ ]:
np.array(wavecar1.band_energy)